In [ ]:
#If th openai api key is not set already, please set it here.
import os
if 'OPENAI_API_KEY' not in os.environ:  
     os.environ['OPENAI_API_KEY'] = 'your key'

In [ ]:
from amadeusgpt.main import AMADEUS
from amadeusgpt.config import Config
import amadeusgpt
from pathlib import Path
import matplotlib.pyplot as plt
import cv2

In [ ]:
video_path = "/mnt/md0/shaokai/mouse_vid.mpg"
scene_frame_number = 400
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, scene_frame_number)
    # Read the frame
    ret, image = cap.read()

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cap.release()
plt.imshow(image)
    

### Note that unlike other notebooks, we don't have keypoint_file_path here (as it's not provided)
### By default, we use gpt-4o to determine which SuperAnimal models to run and it will run SuperAnimal in the first time the keypoints related queries are asked
### Make sure you use a short video clips if you are not using GPUs in Linux (Mac silicon support to be added)

In [ ]:
amadeus_root = Path(amadeusgpt.__file__).parent.parent
config = Config( amadeus_root / "amadeusgpt/configs/template.yaml")
config['video_info']['video_file_path'] = video_path
config['video_info']['scene_frame_number'] = scene_frame_number
print (config)
amadeus = AMADEUS(config)

In [ ]:
from IPython.display import Markdown, display
def parse_result(amadeus, qa_message):
    display(Markdown(qa_message['chain_of_thought']))
    sandbox = amadeus.sandbox
    qa_message = sandbox.code_execution(qa_message)
    sandbox.render_qa_message(qa_message)
    print ('after executing the function')
    display(qa_message['meta_info'])
    print(qa_message['pose_video'])
    display(Markdown(str(qa_message['function_rets'])))

In [ ]:
query = "Plot the trajectory of the animal using the animal center and color it by time"
qa_message = amadeus.step(query)
parse_result(amadeus, qa_message)

In [ ]:
query = "give me the occupancy plot using the animal center"
qa_message = amadeus.step(query)
parse_result(amadeus, qa_message)